In [ ]:
'''
Here we will move 'A' within a 4x4 map till it reaches the start and end terminal with the help of
dynamic programming and reinforcement learning.
'''

In [1]:
#import
from random import random, choice
from copy import deepcopy

In [2]:
# Map Example: 4 x 4
#    -----------------
#    | 0 | 1 | 2 | 3 |
#    -----------------
#    | 4 | 5 | 6 | 7 |
#    -----------------
#    | A | 9 | 10| 11|
#    -----------------
#    | 12| 13| 14| 15|
#    -----------------
# We are defining a 4x4 map with terminals at 0 and 15
dimensions = (r,c)=(4,4)
MAP_RANGE = r*c
positions = range(MAP_RANGE) #Include all values except initial and final (0 and 16)
TERMINALS = {0,15}
open_pos = set(positions)- TERMINALS #creates a set of open positions

In [3]:
def create_random_policy():
    #Random Policy - assign each position with a probability
    #Every position is given the probability of 0.25 except the terminals which have the probaility 0
    return{i: {d: (0 if i in TERMINALS else 0.25) for d in "NEWS"} for i in positions}

#print(create_random_policy())

In [4]:
def in_range(i, step):
    #here i is current position and c is number of columns
    if step =='N' and i -c >=0: 
        i -=c
    elif step =='E' and i % c != c-1:
        i += 1
    elif step == 'W' and i%c != 0:
        i -=1
    elif step == 'S' and i+c <MAP_RANGE:
        i +=c
    return i

In [5]:
def create_position_change_map():
    #Position Change Map: returns how the agent will move within the map
    #The movement will not exceed borders and the agent will remian in place if asked to do so
    return {i: {d: in_range(i, d) for d in "NEWS"} for i in positions}

#print(create_position_change_map())

In [6]:
#Print Board/MAP
def print_board(agnt_pos):
    hline = "-" * (4 * c + 1) #horizontal line
    board = [hline] #initialize board
    for i in range(0, MAP_RANGE, c):
        board.append(''.join(("| " + ('A' if c == agnt_pos else 'X'
                                      if c in TERMINALS else ' ') + ' ')
                             for c in positions[i:i+c]) + '|')
        board.append(hline)
    print('\n'.join(board), "\n")
    
#print_board(3)

In [7]:
def agent(policy, start_pos=None, verbose=False):
    # Choose initial position
    agent_pos = choice(positions) if start_pos is None else start_pos #passes a random starting point if not specified
    state_to_state_prime = create_position_change_map() 

    step_no = 0
    if verbose:
        print(f"Step: {step_no} Position: {agent_pos} Action: Initial")
        print_board(agent_pos)
    # While agent is not at terminal postitions, let it move.
    while agent_pos not in TERMINALS:
        next_move = random()
        lower_bound = 0
        for action, chance in policy[agent_pos].items():
            if lower_bound <= next_move < lower_bound + chance:
                agent_pos = state_to_state_prime[agent_pos][action]
                break
            lower_bound += chance
        step_no += 1

        if verbose:
            print(f"Step: {step_no} Position: {agent_pos} Action: {action}")
            print_board(agent_pos)

    return step_no   

In [8]:
policy = create_random_policy()
#agent(policy, None, verbose=True) #uncomment it to see the movement of agent within the map till it reaches a terminal

data = [agent(policy) for i in range(1000)] 
#runs the agent function with random starting values for 1000 times and stores number of steps required in form of a list
print(f"Random Policy\n-Average steps to finish: {sum(data) / len(data)}")


Random Policy
-Average steps to finish: 16.155


In [9]:
def create_probability_map():
    pos_map = create_position_change_map()
    return {(pr, stt, mv): 0 if pr != pos_map[stt][mv] else 1
            for stt in positions for mv in 'NEWS' for pr in positions}
    #returns list of all possible movements
    #E.g. (6,2,'S'):1 states that movement of 2 towards 6 in the south direction is a valid move

#create_probability_map()

In [10]:
def create_greedy_policy(v_s):
    pos_map = create_position_change_map()
    policy = {}

    for stt in positions:
        state_values = {d: v_s[pos_map[stt][d]] for d in 'NEWS'}
        max_st_val = max(state_values.values())
        max_actions = [k for k, v in state_values.items() if v == max_st_val]
        policy[stt] = {d: 1 / len(max_actions) if d in max_actions and
                       stt not in TERMINALS else 0.0 for d in 'NEWS'}
    return policy


In [11]:
def iterative_policy_evaluation(policy, theta=0.01, discount=0.5):
    v_s = {i: 0 for i in positions}
    prob_map = create_probability_map()

    delta = 100
    while delta >= theta:
        new_vs = {stt: round(sum(policy[stt][mv]
                                 * sum(prob_map[(pr, stt, mv)] *
                                       (-1 + discount * v_s[pr])
                                       for pr in positions)
                                 for mv in "NEWS"), 4) for stt in positions}
        delta = max(abs(v_s[stt] - new_vs[stt]) for stt in positions)
        v_s = deepcopy(new_vs)
    return v_s

In [12]:
v_s = iterative_policy_evaluation(create_random_policy())
policy = create_greedy_policy(v_s)
print(v_s)

data = [agent(policy) for i in range(1000)]

print(f"Greedy Policy V1\nAverage steps to finish: {sum(data) / len(data)}")

{0: 0.0, 1: -1.6914, 2: -1.944, 3: -1.9773, 4: -1.6914, 5: -1.9107, 6: -1.961, 7: -1.944, 8: -1.944, 9: -1.961, 10: -1.9107, 11: -1.6914, 12: -1.9773, 13: -1.944, 14: -1.6914, 15: 0.0}
Greedy Policy V1
Average steps to finish: 1.763


In [13]:
v_s = iterative_policy_evaluation(policy)
policy = create_greedy_policy(v_s)
print(v_s)

data = [agent(policy) for i in range(1000)]
print(f"Greedy Policy V2\nAverage steps to finish: {sum(data) / len(data)}")


{0: 0.0, 1: -1.0, 2: -1.5, 3: -1.75, 4: -1.0, 5: -1.5, 6: -1.75, 7: -1.5, 8: -1.5, 9: -1.75, 10: -1.5, 11: -1.0, 12: -1.75, 13: -1.5, 14: -1.0, 15: 0.0}
Greedy Policy V2
Average steps to finish: 1.793


In [14]:
agent(policy, verbose=True)

Step: 0 Position: 1 Action: Initial
-----------------
| X | A |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | X |
----------------- 

Step: 1 Position: 0 Action: W
-----------------
| A |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | X |
----------------- 



1

In [15]:
def value_iteration(v_s=None, theta=0.01, discount=0.5):
    v_s = {i: 0 for i in positions} if v_s is None else v_s
    prob_map = create_probability_map()

    delta = 100
    while delta >= theta:
        new_vs = {stt: round(max(sum(prob_map[(pr, stt, mv)]
                                     * (-1 + discount * v_s[pr])
                                     for pr in positions) for mv in "NEWS"), 4)
                  for stt in positions}
        delta = max(abs(v_s[stt] - new_vs[stt]) for stt in positions)
        v_s = deepcopy(new_vs)
    return v_s


v_s = value_iteration()
policy = create_greedy_policy(v_s)

print(v_s)

data = [agent(policy) for i in range(1000)]

print("Greedy Policy with Value Iteration\nAverage steps to finish:",
      f"{sum(data) / len(data)}")


{0: -1.9922, 1: -1.9922, 2: -1.9922, 3: -1.9922, 4: -1.9922, 5: -1.9922, 6: -1.9922, 7: -1.9922, 8: -1.9922, 9: -1.9922, 10: -1.9922, 11: -1.9922, 12: -1.9922, 13: -1.9922, 14: -1.9922, 15: -1.9922}
Greedy Policy with Value Iteration
Average steps to finish: 16.485


In [16]:
agent(create_greedy_policy(value_iteration(v_s)), verbose=1)

Step: 0 Position: 8 Action: Initial
-----------------
| X |   |   |   |
-----------------
|   |   |   |   |
-----------------
| A |   |   |   |
-----------------
|   |   |   | X |
----------------- 

Step: 1 Position: 4 Action: N
-----------------
| X |   |   |   |
-----------------
| A |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | X |
----------------- 

Step: 2 Position: 0 Action: N
-----------------
| A |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   |   |
-----------------
|   |   |   | X |
----------------- 



2